
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 自動並列処理

:label: `sec_auto_para`

深層学習フレームワーク (MXNet や PyTorch など) は、バックエンドで計算グラフを自動的に構築します。計算グラフを使用すると、システムはすべての依存関係を認識し、相互依存しない複数のタスクを選択的に並列実行して速度を向上させることができます。たとえば、:numref: `fig_asyncgraph`の :numref: `sec_async`は 2 つの変数を独立して初期化します。したがって、システムはそれらを並行して実行することを選択できます。

通常、1 人のオペレーターがすべての CPU または 1 つの GPU 上のすべての計算リソースを使用します。たとえば、 `dot`演算子は、単一のマシン上に複数の CPU プロセッサがある場合でも、すべての CPU 上のすべてのコア (およびスレッド) を使用します。同じことが単一の GPU にも当てはまります。したがって、並列化は単一デバイスのコンピューターではあまり役に立ちません。複数のデバイスを使用すると、さらに重要になります。通常、並列化は複数の GPU 間で最も関連性が高くなりますが、ローカル CPU を追加するとパフォーマンスがわずかに向上します。たとえば、GPU と CPU を組み合わせたコンピューター ビジョン モデルのトレーニングに焦点を当てた :citet: `Hadjis.Zhang.Mitliagkas.ea.2016`を参照してください。自動的に並列化されるフレームワークの利便性を利用して、数行の Python コードで同じ目標を達成できます。より広範に、自動並列計算に関する説明は、CPU と GPU の両方を使用した並列計算、および計算と通信の並列化に焦点を当てています。

このセクションの実験を実行するには、少なくとも 2 つの GPU が必要であることに注意してください。


In [1]:
import torch
from d2l import torch as d2l


## GPU での並列計算

テストする参照ワークロードを定義することから始めましょう。以下の`run`関数は、2 つの変数`x_gpu1`と`x_gpu2`に割り当てられたデータを使用して、選択したデバイス上で 10 回の行列間の乗算を実行します。


In [2]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])


次に、関数をデータに適用します。キャッシュが結果に影響を及ぼさないようにするため、測定前にデバイスのいずれかで 1 回のパスを実行してデバイスをウォームアップします。 `torch.cuda.synchronize()` CUDA デバイス上のすべてのストリームのすべてのカーネルが完了するのを待ちます。これは`device`引数、つまり同期する必要があるデバイスを受け取ります。 device 引数が`None` (デフォルト) の場合、 `current_device()`で指定された現在のデバイスを使用します。


In [3]:
run(x_gpu1)
run(x_gpu2)  # Warm-up all devices
torch.cuda.synchronize(devices[0])
torch.cuda.synchronize(devices[1])

with d2l.Benchmark('GPU1 time'):
    run(x_gpu1)
    torch.cuda.synchronize(devices[0])

with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])

GPU1 time: 0.4967 sec


GPU2 time: 0.5151 sec



両方のタスク間の`synchronize`ステートメントを削除すると、システムは両方のデバイスで自動的に計算を自由に並列化できるようになります。


In [4]:
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()

GPU1 & GPU2: 0.5000 sec



上記の場合、ユーザーに代わって高度なコードを必要とせずに、深層学習フレームワークが両方の GPU デバイスでの計算を自動的にスケジュールするため、合計実行時間はその部分の合計よりも短くなります。

## 並列計算と通信

多くの場合、異なるデバイス間、たとえば CPU と GPU 間、または異なる GPU 間でデータを移動する必要があります。たとえば、これは、複数のアクセラレータ カードにわたる勾配を集約する必要がある分散最適化を実行するときに発生します。 GPU で計算し、結果を CPU にコピーして戻すことで、これをシミュレートしてみましょう。


In [5]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

with d2l.Benchmark('Run on GPU1'):
    y = run(x_gpu1)
    torch.cuda.synchronize()

with d2l.Benchmark('Copy to CPU'):
    y_cpu = copy_to_cpu(y)
    torch.cuda.synchronize()

Run on GPU1: 0.5019 sec


Copy to CPU: 2.7168 sec



これはやや非効率的です。リストの残りの部分がまだ計算されている間に、すでに`y`の一部の CPU へのコピーを開始できることに注意してください。この状況は、たとえばミニバッチで (バックプロップ) 勾配を計算するときに発生します。一部のパラメータの勾配は、他のパラメータよりも早く利用可能になります。したがって、GPU がまだ動作している間に PCI-Express バス帯域幅の使用を開始することが有利に機能します。 PyTorch では、 `to()`や`copy_()`などのいくつかの関数は、明示的な`non_blocking`引数を許可します。これにより、呼び出し元は不要な場合に同期をバイパスできます。 `non_blocking=True`を設定すると、このシナリオをシミュレートできるようになります。


In [6]:
with d2l.Benchmark('Run on GPU1 and copy to CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, True)
    torch.cuda.synchronize()

Run on GPU1 and copy to CPU: 2.4682 sec



両方の操作に必要な合計時間は、(予想どおり) それぞれの部分の合計よりも短くなります。このタスクは別のリソース (CPU と GPU 間のバス) を使用するため、並列計算とは異なることに注意してください。実際、両方のデバイスで同時に計算し、通信することができました。上で述べたように、計算と通信の間には依存関係があります。y `y[i]`は、CPU にコピーされる前に計算される必要があります。幸いなことに、システムは y `y[i]` `y[i-1]`をコピーして、合計実行時間を短縮できます。

最後に、:numref: `fig_twogpu`に示すように、CPU と 2 つの GPU でトレーニングする場合の単純な 2 層 MLP の計算グラフとその依存関係を示します。この結果生じる並列プログラムを手動でスケジュールするのは非常に困難です。この場合、最適化のためにグラフベースのコンピューティング バックエンドを使用することが有利です。 

![](http://d2l.ai/_images/twogpu.svg) :label: `fig_twogpu`

## まとめ
- 最新のシステムには、複数の GPU や CPU などのさまざまなデバイスが搭載されています。これらは並行して非同期的に使用できます。
- 最新のシステムには、PCI Express、ストレージ (通常はソリッド ステート ドライブまたはネットワーク経由)、ネットワーク帯域幅など、通信用のさまざまなリソースもあります。これらを並列して使用すると、最大の効率が得られます。
- バックエンドは、自動並列計算と通信を通じてパフォーマンスを向上させることができます。

## 演習
1. このセクションで定義された`run`関数では 8 つの操作が実行されました。それらの間に依存関係はありません。実験を設計して、深層学習フレームワークがそれらを自動的に並列実行するかどうかを確認します。
1. 個々のオペレーターのワークロードが十分に小さい場合は、単一の CPU または GPU でも並列化が役立ちます。これを検証するための実験を計画します。
1.  CPU、GPU での並列計算、および両方のデバイス間の通信を使用する実験を設計します。
1.  NVIDIA の[Nsight](https://developer.nvidia.com/nsight-compute-2019_5)などのデバッガーを使用して、コードが効率的であることを確認します。
1. より複雑なデータ依存関係を含む計算タスクを設計し、パフォーマンスを向上させながら正しい結果が得られるかどうかを確認する実験を実行します。



[ディスカッション](https://discuss.d2l.ai/t/1681)
